# Apis keys

In [ ]:
API_KEY = 'gEfxM0qizKupAtxiJisZyfAHc'
SECRET_KEY = 'LTK9PKTdyxtIPLxFVgguThijjwYmjAN4UkwR3panqgW9oWByjL'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAANNMWQEAAAAAseDuRG%2FqCkTwmPokq%2BjNwf9%2BT%2Fs%3DWMUoWVCtHqRqfdK5sNP2LiabJSrsIcjQOf19hL8BlnBXU9gXA6'
ACCESS_TOKEN = '3542954414-euXzt1IRoeg6gYZhb1WF7JgJyB1QTFFf0AFrT7m'
SECRET_TOKEN = '1PGMEabNqgYSx9fIpf0HWkdZqsSVXk3Jd0Ke3zk7oCn1A'

#Importing libraries


In [ ]:
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold
from nltk.stem import PorterStemmer
import tweepy as tw
import pandas as pd
import numpy as np
import nltk
import re


from pickle import load
import pickle

#Reading covid tweets

In [ ]:
auth = tw.OAuthHandler(API_KEY, SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, SECRET_TOKEN)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
search_words = "#COVID19 + -filter:retweets + -filter:links"

In [ ]:
counter=1

for tweet in tw.Cursor(api.search, tweet_mode='extended', q=search_words, lang="en").items():
        print('Tweet Downloaded: ', counter)
        print(tweet.full_text)
        counter+=1
        if counter >= 1000:
            break

Tweet Downloaded:  1
7 new cases in Equatorial Guinea 

[0:44 GMT] #coronavirus #CoronaVirusUpdate #COVID19 #CoronavirusPandemic
Tweet Downloaded:  2
@DrPJLillie @IdVilchez @wfwrighID @DrToddLee @ABsteward @SAIRABT @Boghuma @AntibioticDoc @Darcy_ID_doc @JonathanRyderMD @FungalDoc Curious on #idtwitter how many people are using more dalbavancin these days. National use of drug has been on ascent since just before #COVID19 arrived, dipped during early COVID19, and how has been back on steady ascent.  What has specifically led to more use of agent recently?
Tweet Downloaded:  3
Got word today YET ANOTHER colleague contracted #COVID19. He didn't seem to get a terrible case and is recovering.

He is a parent and his child caught covid already. He's not 100% clear where he got it, but here's a hint.

He lives in TX.

Seems self explanatory to me...
Tweet Downloaded:  4
When is the variant that makes people stop being such scumbags coming out? #COVID19 #covidvariant
Tweet Downloaded:  5
The r

#Creating tweets dataframe

In [ ]:
counter=1

In [ ]:
import pandas as pd


tweet_dataset = pd.DataFrame(columns=['Tweet Id', 'Tweet Date', 'Follower Count', 'Account Verified', 'Favorite Count', 'Retweets', 'Tweet Text'])

for tweet in tw.Cursor(api.search, tweet_mode='extended', q=search_words, lang="en").items():
        print('Tweet Downloaded: ', counter)
        appending_dataframe = pd.DataFrame([[tweet.id, tweet.created_at, tweet.user.followers_count, tweet.user.verified, tweet.favorite_count, tweet.retweet_count, tweet.full_text]], columns=['Tweet Id', 'Tweet Date', 'Follower Count', 'Account Verified', 'Favorite Count', 'Retweets', 'Tweet Text'])
        tweet_dataset = tweet_dataset.append(appending_dataframe)
        counter+=1
        if counter >= 1000:
            break

tweet_dataset.to_csv('tweet_dataset_COVID.csv', index=False)

Tweet Downloaded:  1
Tweet Downloaded:  2
Tweet Downloaded:  3
Tweet Downloaded:  4
Tweet Downloaded:  5
Tweet Downloaded:  6
Tweet Downloaded:  7
Tweet Downloaded:  8
Tweet Downloaded:  9
Tweet Downloaded:  10
Tweet Downloaded:  11
Tweet Downloaded:  12
Tweet Downloaded:  13
Tweet Downloaded:  14
Tweet Downloaded:  15
Tweet Downloaded:  16
Tweet Downloaded:  17
Tweet Downloaded:  18
Tweet Downloaded:  19
Tweet Downloaded:  20
Tweet Downloaded:  21
Tweet Downloaded:  22
Tweet Downloaded:  23
Tweet Downloaded:  24
Tweet Downloaded:  25
Tweet Downloaded:  26
Tweet Downloaded:  27
Tweet Downloaded:  28
Tweet Downloaded:  29
Tweet Downloaded:  30
Tweet Downloaded:  31
Tweet Downloaded:  32
Tweet Downloaded:  33
Tweet Downloaded:  34
Tweet Downloaded:  35
Tweet Downloaded:  36
Tweet Downloaded:  37
Tweet Downloaded:  38
Tweet Downloaded:  39
Tweet Downloaded:  40
Tweet Downloaded:  41
Tweet Downloaded:  42
Tweet Downloaded:  43
Tweet Downloaded:  44
Tweet Downloaded:  45
Tweet Downloaded:  

In [ ]:
df = pd.read_csv('tweet_dataset_COVID.csv')

In [ ]:
df

,Tweet Id,Tweet Date,Follower Count,Account Verified,Favorite Count,Retweets,Tweet Text
0,1466553904896921621,2021-12-02 23:44:32,1255,False,0,0,7 new cases in Equatorial Guinea \n\n[0:44 GMT...
1,1466553859271118852,2021-12-02 23:44:21,2560,True,0,0,@DrPJLillie @IdVilchez @wfwrighID @DrToddLee @...
2,1466553850110701568,2021-12-02 23:44:19,442,False,0,0,Got word today YET ANOTHER colleague contracte...
3,1466553763506778115,2021-12-02 23:43:59,749,False,2,0,When is the variant that makes people stop bei...
4,1466553743265128455,2021-12-02 23:43:54,733,False,0,0,The real Death Panels are\nCongressional Commi...
...,...,...,...,...,...,...,...
994,1466497512257298444,2021-12-02 20:00:27,21,False,0,0,"Today, there have been 2,142 new confirmed cas..."
995,1466497512215355393,2021-12-02 20:00:27,21,False,0,0,"At the moment, there have been 1,741,417 recov..."
996,1466497512202813448,2021-12-02 20:00:27,21,False,0,0,"Currently, there are a total of 1,797,771 conf..."
997,1466497430539616263,2021-12-02 20:00:08,506,False,0,0,UK COVID19 DEATHS\n\n**Within 28 Days Of Posit...


#Preprocessing

In [ ]:
df.columns

Index(['Tweet Id', 'Tweet Date', 'Follower Count', 'Account Verified',
       'Favorite Count', 'Retweets', 'Tweet Text'],
      dtype='object')

In [ ]:
df_new = df.rename(columns={'Tweet Text': 'Text'}, index={'ONE': 'Row_1'})


In [ ]:
df_new= df_new.drop(['Follower Count', 'Account Verified','Favorite Count','Retweets','Tweet Date','Tweet Id'], axis=1)

In [ ]:
df_new.head()

,Text
0,7 new cases in Equatorial Guinea \n\n[0:44 GMT...
1,@DrPJLillie @IdVilchez @wfwrighID @DrToddLee @...
2,Got word today YET ANOTHER colleague contracte...
3,When is the variant that makes people stop bei...
4,The real Death Panels are\nCongressional Commi...


In [ ]:
df_new.columns

Index(['Text'], dtype='object')

#Clases for cleanning 

In [ ]:
class CleanningData():
    def __init__(self):
        pass


    def _scrub_words(self, text):
        text=re.sub("(<.*?>)","",text)# remove html markup
        text=re.sub("(\\W|\\d)"," ",text)#remove non-ascii and digits
        text=text.strip()#remove whitespace

        return text
  
    #Noise removal, stop word removal, normalizing?
    def _cleanString(self, s, special_chars = "\":,.|#ðÿœžðÿâœœïÿœžÿºÿÿœžÿ"):
        s = s.lower()
        "this section is removing hyperparameters ans special ones"
        web_regex =  r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        s = re.sub('(?<!\w)@[\w+]{1,15}', '', s)
        s = re.sub(web_regex, '', s)
        
        for char in special_chars:
            s = s.replace(char, "")
            s = s.replace("\n", "")
            s = self._scrub_words(s)

        tokenizer = TweetTokenizer()
        stop_words = set(stopwords.words('english'))
        cleaned_words = [w for w in tokenizer.tokenize(s) if w not in stop_words]
        
        return " ".join(cleaned_words)


    def _stemWords(self, sentence):
        stemmer, tokenizer = PorterStemmer(), TweetTokenizer()
        stemmed_words = [stemmer.stem(w) for w in tokenizer.tokenize(sentence)]

        return " ".join(stemmed_words)
    



    # Funcion aplication for cleanning data tweets.
    def cleanFrame(self, frame):
        frame['Cleaned_text'] = frame.Text.apply(self._cleanString)

    def stemFrame(self, frame):
        frame['Steammed_Text'] = frame.Cleaned_text.apply(self._stemWords)

    
    def rebrand_senti(self, senti):
        if senti != 0:
            senti = 1
        return senti


# Inicializating Cleaner

In [ ]:
clean = CleanningData()
clean.cleanFrame(df_new)
clean.stemFrame(df_new)

#df_new.senti = df.senti.apply(clr.rebrand_senti)
df_new.head()

,Text,Cleaned_text,Steammed_Text
0,7 new cases in Equatorial Guinea \n\n[0:44 GMT...,new cases equatorial guinea gmt coronavirus co...,new case equatori guinea gmt coronaviru corona...
1,@DrPJLillie @IdVilchez @wfwrighID @DrToddLee @...,curious idtwitter many people using dalbavanci...,curiou idtwitt mani peopl use dalbavancin day ...
2,Got word today YET ANOTHER colleague contracte...,got word today yet another colleague contracte...,got word today yet anoth colleagu contract cov...
3,When is the variant that makes people stop bei...,variant makes people stop scumbags coming covi...,variant make peopl stop scumbag come covid cov...
4,The real Death Panels are\nCongressional Commi...,real death panels arecongressional committeesc...,real death panel arecongression committeeschai...


In [ ]:
import nltk
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('words')
nltk.download('vader_lexicon')

words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
sid = SentimentIntensityAnalyzer()

list1 = []

for i in df_new['Steammed_Text']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

In [ ]:
dflist = pd.DataFrame(list1)

In [ ]:
labelScore = list(dflist.sum(axis = 1, skipna = True))
newLabelScore = [1 if (x>0) else 0 for x in labelScore]

In [ ]:
df_new['polarity']= newLabelScore
df_new.head(3)

,Text,Cleaned_text,Steammed_Text,polarity
0,7 new cases in Equatorial Guinea \n\n[0:44 GMT...,new cases equatorial guinea gmt coronavirus co...,new case equatori guinea gmt coronaviru corona...,0
1,@DrPJLillie @IdVilchez @wfwrighID @DrToddLee @...,curious idtwitter many people using dalbavanci...,curiou idtwitt mani peopl use dalbavancin day ...,0
2,Got word today YET ANOTHER colleague contracte...,got word today yet another colleague contracte...,got word today yet anoth colleagu contract cov...,1


In [ ]:
imp_df = df_new[['Steammed_Text','polarity']]

In [ ]:
imp_df.head()

,Steammed_Text,polarity
0,new case equatori guinea gmt coronaviru corona...,0
1,curiou idtwitt mani peopl use dalbavancin day ...,0
2,got word today yet anoth colleagu contract cov...,1
3,variant make peopl stop scumbag come covid cov...,0
4,real death panel arecongression committeeschai...,0


#Preprocesing Data 
### (preparing data for the model)

In [ ]:
def get_vocab(df):
    vector = []
    tweets  = ' '.join(list(imp_df.Steammed_Text))
    stop_words = set(stopwords.words('english'))

    for word in tweets.split():

        if (word not in vector) and (word not in stop_words): 
            vector.append(word)

    return vector

vector = get_vocab(df)

print(len(vector))

3835


In [ ]:
def word_frequencies(df, vector):

    pos_fr = dict.fromkeys(vector, 0)
    neg_fr = dict.fromkeys(vector, 0)

    for i in range(df.shape[0]):

        word_list = (df.iloc[i].Steammed_Text).split(' ')
        sent = df.iloc[i].polarity

        for word in word_list:

            if word in vector:

                if sent == 0:

                    neg_fr[word] += 1
                else:
                    pos_fr[word] += 1
                    
    return (pos_fr, neg_fr)

In [ ]:
pos_dict, neg_dict = word_frequencies(imp_df,vector)

In [ ]:
def transform_data(df, pos_dict, neg_dict, vector):

    new_df = {'bias':[], 'positivity':[], 'negativity':[], 'target_senti':[]}
    stemed_sentences = df.Steammed_Text
   
    for sentence in stemed_sentences:
        positivity = 0
        negativity = 0
        words = sentence.split()

        #count positivity
        for word in words:
          
            if word in vector:
                positivity += pos_dict[word]
                negativity += neg_dict[word]

        new_df['bias'].append(1)

        new_df['positivity'].append(positivity)
        new_df['negativity'].append(negativity)
        new_df['target_senti'] = df.polarity

    return pd.DataFrame(new_df)    


In [ ]:
trans_df = transform_data(imp_df, pos_dict, neg_dict, vector)
trans_df.head()

,bias,positivity,negativity,target_senti
0,1,502,1222,0
1,1,1035,1765,0
2,1,1118,1926,1
3,1,580,976,0
4,1,468,933,0


In [ ]:
trans_df.target_senti.unique()

array([0, 1])

In [ ]:
kfold = KFold(n_splits = 5, shuffle = True)

In [ ]:
trans_df.iloc[['1','2']]

,bias,positivity,negativity,target_senti
1,1,1035,1765,0
2,1,1118,1926,1


In [ ]:
X = trans_df[['bias', 'positivity', 'negativity']]
Y = trans_df.target_senti

avg = 0
avg_f1 = 0
avg_rec = 0

# Importing and applying model logistic regression

In [ ]:
loaded_model = pickle.load(open('/content/modeLr.sav', 'rb'))
result = loaded_model.score(X, Y)

##### predictions

In [ ]:
predictions = loaded_model.predict(X)

# accuracy

In [ ]:
result

0.6516516516516516

In [ ]:
len(prediction)

999

In [ ]:
last_df = pd.DataFrame(df_new['Text'])
last_df['id_tweet'] = df['Tweet Id']
last_df['Tweet_Date'] = df['Tweet Date']
last_df['Follower_count'] = df['Follower Count']
last_df['cleaned_text'] = df_new['Steammed_Text']


#Final dataframe for visualization dashboards on django

In [ ]:
last_df['prediction'] = predictions

In [ ]:
last_df.head()

,Text,id_tweet,Tweet_Date,Follower_count,cleaned_text,prediction
0,7 new cases in Equatorial Guinea \n\n[0:44 GMT...,1466553904896921621,2021-12-02 23:44:32,1255,new case equatori guinea gmt coronaviru corona...,0
1,@DrPJLillie @IdVilchez @wfwrighID @DrToddLee @...,1466553859271118852,2021-12-02 23:44:21,2560,curiou idtwitt mani peopl use dalbavancin day ...,0
2,Got word today YET ANOTHER colleague contracte...,1466553850110701568,2021-12-02 23:44:19,442,got word today yet anoth colleagu contract cov...,0
3,When is the variant that makes people stop bei...,1466553763506778115,2021-12-02 23:43:59,749,variant make peopl stop scumbag come covid cov...,0
4,The real Death Panels are\nCongressional Commi...,1466553743265128455,2021-12-02 23:43:54,733,real death panel arecongression committeeschai...,0


In [ ]:
last_df.prediction.unique()

array([0, 1])